# Reddit - CryptoCurrencies Data Collection

## Objective

Get sub reddit information for the following CryptoCurrencies
*['Bitcoin','Ethereum','XRP','EOS', 'Litecoin', 'BTC', 'Tether', 'Stellar', 'TRON', 'Binance']*

1. Whe have used pushshift api to get subreddit information 
2. The data is collected from 1st January 2018 till 27th Feb 2019 [1 Year] for all 10 CryptoCurrencies
3. Then Vader NLTK Library is used to extract sentiment based on the titles of the subreddit
4. Finally we collect sentiment data, titles for for all CryptoCurrencies hourly

In [5]:
#Reddit News Information
import requests
import json
import pprint
import pandas as pd
from dateutil import parser
import datetime
import time
import urllib

def request_API(after,before,sort_type,sort,limit,subreddit,total_requests):
    url = 'https://api.pushshift.io/reddit/submission/search/?'
    headers = {'Accept' : 'application/json', 'Content-Type' : 'application/x-www-form-urlencoded'}
    f = {
        "after": after,
        "before": before,
        "sort_type": sort_type,
        "sort": sort,
        "limit": limit,
        "subreddit": subreddit
        }
    params = urllib.parse.urlencode(f)
    r = requests.get(url,params=params,headers=headers)
    return r

def convert_to_dataframe(api_result):
    api_response =pd.DataFrame.from_dict(get_values['data'])
    cols = ['author','created_utc','domain',
            'full_link','id','num_comments','score','selftext',
            'subreddit','subreddit_id','title','url']
    dictionary_to_df = df[cols]
    return dictionary_to_df


cols = ['author','created_utc','domain',
            'full_link','id','num_comments','score','selftext',
            'subreddit','subreddit_id','title','url']

total_requests = 0
df = pd.DataFrame(columns = cols)
sort_type="created_utc"
sort="desc"
limit=1000
subreddit="crypto"
i = 1
reddits = ['Binance']
#reddits = ['CryptoCurrency','news']
#reddits = ['Bitcoin','Ethereum','XRP','EOS', 'Litecoin', 'BTC', 'Tether', 'Stellar', 'TRON', 'Binance']
for subreddit in reddits:
    requested_min_limit = datetime.datetime.utcnow().timestamp()
    required_time_limit = parser.parse('2018-01-01T00:00:00Z').timestamp()
    while (requested_min_limit > required_time_limit ):
        after=int(required_time_limit)
        before=int(requested_min_limit)
        api_result = request_API(after,before,sort_type,sort,limit,subreddit,total_requests).json()
        if len(api_result["data"]) > 0:
            filename = "Reddit_News_1/"+subreddit+"_"+str(i)+".json"
            with open(filename, 'w') as f:
                    json.dump(api_result,f)
            requested_min_limit = api_result['data'][-1]["created_utc"]
            print('\x1b[1;31m',subreddit,' subreddit data till ',requested_min_limit,' has been downloaded in iteration ',i , '\x1b[0m')
            i = i +1
        else:
            break

 Binance  subreddit data till  1544150661  has been downloaded in iteration  1 
 Binance  subreddit data till  1538092251  has been downloaded in iteration  2 
 Binance  subreddit data till  1532728545  has been downloaded in iteration  3 
 Binance  subreddit data till  1527775292  has been downloaded in iteration  4 
 Binance  subreddit data till  1523672274  has been downloaded in iteration  5 
 Binance  subreddit data till  1516489337  has been downloaded in iteration  6 
 Binance  subreddit data till  1515819084  has been downloaded in iteration  7 
 Binance  subreddit data till  1515478726  has been downloaded in iteration  8 
 Binance  subreddit data till  1515196203  has been downloaded in iteration  9 
 Binance  subreddit data till  1515009705  has been downloaded in iteration  10 
 Binance  subreddit data till  1514765043  has been downloaded in iteration  11 


In [6]:
# Convert all json downloaded files to pandas df
import pandas as pd
from os import listdir
from os.path import isfile, join
mypath = 'reddit_files'

complete_df=pd.DataFrame()

cols = ['author','created_utc','domain',
            'full_link','id','num_comments','score','selftext',
            'subreddit','subreddit_id','title','url']

def unpack_dictionary_columns(df, columns, fillna=None):
    ret = None
    for column in columns:
        if fillna is None:
            ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
            del ret[column]
        else:
            ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
            del ret[column]
            df = ret
    return ret

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for file_name in onlyfiles:
    path_file_name = "reddit_files/"+file_name
    df = pd.read_json(path_file_name)
    complete_df=complete_df.append(df,ignore_index=True)
complete_df = unpack_dictionary_columns(complete_df,['data'])
complete_df = complete_df[cols]
complete_df['created_utc'] = pd.to_datetime(complete_df['created_utc'],unit='s')
complete_df.head(2)

,author,created_utc,domain,full_link,id,num_comments,score,selftext,subreddit,subreddit_id,title,url
0,il_duomino,2018-03-06 03:20:39,i.imgur.com,https://www.reddit.com/r/CryptoCurrency/commen...,82byr1,4,0,,CryptoCurrency,t5_2wlj3,What is going on with Bitcoin Diamond?,https://i.imgur.com/uWE3Y8l.png
1,KnowledgeSuttra,2018-03-06 03:18:40,knowledgesuttra.com,https://www.reddit.com/r/CryptoCurrency/commen...,82byce,0,1,,CryptoCurrency,t5_2wlj3,NEO Coin : A Ethereum killer,http://www.knowledgesuttra.com/blog/neo-coin-e...


In [16]:
print('\x1b[1;31m Total number of reddit pages downloaded \x1b[0m',len(complete_df))

 Total number of reddit pages downloaded  1284023


In [9]:
#Converting Titles to Sentiment Score
import nltk # be sure to have stopwords installed for this using nltk.download_shell()
import pandas as pd 
import string
import sqlite3
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
nltk.download('vader_lexicon')

#conn = sqlite3.connect("CMPT733.db")
#c = conn.cursor()

sia = SIA()
sentiment = complete_df['title'].apply(lambda x : sia.polarity_scores(x))
complete_df=pd.concat([complete_df,sentiment.apply(pd.Series)],1)
complete_df.head(20)
#complete_df.to_sql('TBProcessedRedditNews', con=conn, if_exists='append')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/abejju/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,author,created_utc,domain,full_link,id,num_comments,score,selftext,subreddit,subreddit_id,title,url,compound,neg,neu,pos
0,il_duomino,2018-03-06 03:20:39,i.imgur.com,https://www.reddit.com/r/CryptoCurrency/commen...,82byr1,4,0,,CryptoCurrency,t5_2wlj3,What is going on with Bitcoin Diamond?,https://i.imgur.com/uWE3Y8l.png,0.3400,0.000,0.714,0.286
1,KnowledgeSuttra,2018-03-06 03:18:40,knowledgesuttra.com,https://www.reddit.com/r/CryptoCurrency/commen...,82byce,0,1,,CryptoCurrency,t5_2wlj3,NEO Coin : A Ethereum killer,http://www.knowledgesuttra.com/blog/neo-coin-e...,-0.6486,0.589,0.411,0.000
2,MaxMusa,2018-03-06 03:16:27,self.CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,82bxu9,84,147,"Am I missing something, or is this essentially...",CryptoCurrency,t5_2wlj3,Why is nobody talking about TrueUSD being list...,https://www.reddit.com/r/CryptoCurrency/commen...,0.0772,0.073,0.826,0.100
3,MediumBaller,2018-03-06 03:15:34,sosnc.gov,https://www.reddit.com/r/CryptoCurrency/commen...,82bxn0,1,1,,CryptoCurrency,t5_2wlj3,"Cease and Desist Filed Against ""Power Mining P...",https://www.sosnc.gov/vs2010/Soskb.Web.Content...,0.2960,0.000,0.845,0.155
4,ColinRoss,2018-03-06 03:15:07,ci.covesting.io,https://www.reddit.com/r/CryptoCurrency/commen...,82bxjq,0,1,,CryptoCurrency,t5_2wlj3,Moscow Opens First Crypto Mining Equipment Store,https://ci.covesting.io/article/first-cryptocu...,0.0000,0.000,1.000,0.000
5,Kpenney,2018-03-06 03:11:16,self.CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,82bwns,30,0,Why even put trust in them? XRP is inarguably ...,CryptoCurrency,t5_2wlj3,IF coinbase doesn't accept XRP as a tradepair...,https://www.reddit.com/r/CryptoCurrency/commen...,-0.2924,0.267,0.733,0.000
6,Dexter63,2018-03-06 03:10:14,self.CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,82bwf3,0,1,[removed],CryptoCurrency,t5_2wlj3,South Korea busting out of Crypto Coop?,https://www.reddit.com/r/CryptoCurrency/commen...,0.0000,0.000,1.000,0.000
7,ipizi,2018-03-06 03:08:53,self.CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,82bw3h,1,0,"Okay, yes typically I can read buy/sell orders...",CryptoCurrency,t5_2wlj3,"In Tidex, what does the ""sum"" column mean?",https://www.reddit.com/r/CryptoCurrency/commen...,0.0000,0.000,1.000,0.000
8,doebro,2018-03-06 03:06:34,i.redd.it,https://www.reddit.com/r/CryptoCurrency/commen...,82bvlf,13,19,,CryptoCurrency,t5_2wlj3,OG altcoins,https://i.redd.it/7gv51gck72k01.jpg,0.0000,0.000,1.000,0.000
9,raveiskingcom,2018-03-06 03:05:22,soundcloud.com,https://www.reddit.com/r/CryptoCurrency/commen...,82bvbi,8,0,,CryptoCurrency,t5_2wlj3,"Wow, the Let's Talk Bitcoin crew discuss gende...",https://soundcloud.com/mindtomatter/ltb-358-th...,0.5859,0.000,0.868,0.132


In [10]:
import pandas as pd
from dateutil import parser
import datetime
import time
import sqlite3
conn = sqlite3.connect("CMPT733.db")
c = conn.cursor()

df = pd.read_sql("SELECT * FROM TBProcessedRedditNews where subreddit not in ('CryptoCurrency','news')", conn)
df['created_utc'] = pd.to_datetime(df['created_utc'],infer_datetime_format=True)

ap = df.set_index('created_utc').groupby(['subreddit',pd.TimeGrouper('H')]).mean()
qp = df.set_index('created_utc').groupby(['subreddit',pd.TimeGrouper('H')]).count()
ap = ap.reset_index()
qp = qp.reset_index()
def row_to_list(x,col_name):
    return list(x[col_name])

fp = df.set_index('created_utc').groupby(['subreddit',pd.TimeGrouper('H')]).apply(lambda x: row_to_list(x,'title')).reset_index(name='title').dropna()
kp = df.set_index('created_utc').groupby(['subreddit',pd.TimeGrouper('H')]).apply(lambda x: row_to_list(x,'domain')).reset_index(name='domain').dropna()

ap = ap[['subreddit','created_utc','score','compound','neg','neu','pos']]
qp = qp[['subreddit','created_utc','num_comments']]

result = pd.merge(ap,qp, how='inner',on=['subreddit','created_utc'])
result = pd.merge(result,fp, how='inner',on=['subreddit','created_utc'])
result = pd.merge(result,kp, how='inner',on=['subreddit','created_utc'])

#result.to_json('reddit_processed_news.json',orient='records',date_format='iso')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)


In [11]:
result

,subreddit,created_utc,score,compound,neg,neu,pos,num_comments,title,domain
0,Bitcoin,2018-01-01 00:00:00,4.729730,0.085327,0.076351,0.802243,0.121378,37,"[Using specific UTXO to send coins in Trezor?,...","[self.Bitcoin, self.Bitcoin, reviewcoin.me, ho..."
1,Bitcoin,2018-01-01 01:00:00,14.238095,-0.011038,0.064714,0.880143,0.055143,21,"[Doing some shopping here in Tokyo, Japan 😍, I...","[i.redd.it, self.Bitcoin, self.Bitcoin, self.B..."
2,Bitcoin,2018-01-01 02:00:00,4.171429,0.058691,0.065771,0.815114,0.119114,35,"[&lt; ADX (AdEx) &gt; New Hot Altcoin, How far...","[i.redd.it, self.Bitcoin, reviewcoin.me, self...."
3,Bitcoin,2018-01-01 03:00:00,2.200000,0.070145,0.088150,0.797650,0.114200,20,"[21k ATH by midnight or...?, BTC-e hacked Mt. ...","[self.Bitcoin, self.Bitcoin, self.Bitcoin, sel..."
4,Bitcoin,2018-01-01 04:00:00,1.958333,0.039850,0.068083,0.834042,0.097833,24,[Unable to login Blockchain.info (1 bitcoin in...,"[self.Bitcoin, self.Bitcoin, self.Bitcoin, sel..."
5,Bitcoin,2018-01-01 05:00:00,422.619048,0.066924,0.027048,0.887381,0.085571,21,[How do we know altcoins won't use the lightni...,"[self.Bitcoin, self.Bitcoin, self.Bitcoin, bre..."
6,Bitcoin,2018-01-01 06:00:00,43.538462,0.193085,0.005077,0.913769,0.081231,13,"[flight4bit, Found strong clue about Satoshi N...","[self.Bitcoin, self.Bitcoin, self.Bitcoin, ind..."
7,Bitcoin,2018-01-01 07:00:00,10.882353,0.234900,0.055529,0.688235,0.256235,17,"[Mark of the Bea I mean bit.., Ripple. Does th...","[i.redd.it, self.Bitcoin, binance.com, self.Bi..."
8,Bitcoin,2018-01-01 08:00:00,35.428571,-0.105186,0.107857,0.830524,0.061619,21,"[Exodus wallet security questions., Twitch str...","[self.Bitcoin, self.Bitcoin, self.Bitcoin, alt..."
9,Bitcoin,2018-01-01 09:00:00,97.923077,0.061362,0.097462,0.780462,0.122231,13,"[So wussup with the Bitpay fees?, DW about Bit...","[self.Bitcoin, e-tremula.blogspot.com, self.Bi..."


In [2]:
# Store the processed data in json
import pandas as pd
from dateutil import parser
import datetime
import time
import sqlite3
conn = sqlite3.connect("CMPT733.db")
c = conn.cursor()

result = pd.read_json('reddit_processed_news.json')
result['created_utc'] = pd.to_datetime(result['created_utc'],infer_datetime_format=True)

In [17]:
# Final Processed DF
result.head(5)

,compound,created_utc,domain,neg,neu,num_comments,pos,score,subreddit,title
0,0.085327,2018-01-01 00:00:00+00:00,"[self.Bitcoin, self.Bitcoin, reviewcoin.me, ho...",0.076351,0.802243,37,0.121378,4.729730,Bitcoin,"[Using specific UTXO to send coins in Trezor?,..."
1,-0.011038,2018-01-01 01:00:00+00:00,"[i.redd.it, self.Bitcoin, self.Bitcoin, self.B...",0.064714,0.880143,21,0.055143,14.238095,Bitcoin,"[Doing some shopping here in Tokyo, Japan 😍, I..."
2,0.058691,2018-01-01 02:00:00+00:00,"[i.redd.it, self.Bitcoin, reviewcoin.me, self....",0.065771,0.815114,35,0.119114,4.171429,Bitcoin,"[&lt; ADX (AdEx) &gt; New Hot Altcoin, How far..."
3,0.070145,2018-01-01 03:00:00+00:00,"[self.Bitcoin, self.Bitcoin, self.Bitcoin, sel...",0.088150,0.797650,20,0.114200,2.200000,Bitcoin,"[21k ATH by midnight or...?, BTC-e hacked Mt. ..."
4,0.039850,2018-01-01 04:00:00+00:00,"[self.Bitcoin, self.Bitcoin, self.Bitcoin, sel...",0.068083,0.834042,24,0.097833,1.958333,Bitcoin,[Unable to login Blockchain.info (1 bitcoin in...


In [3]:
analysis = result[['compound','subreddit']]
positive_df = analysis.rename({"compound":"positive_news"},axis='columns')
negative_df = analysis.rename({"compound":"negative_news"},axis='columns')

In [4]:
positive_df = positive_df[positive_df.positive_news > 0]
positive_df = positive_df.groupby(['subreddit']).count().reset_index()

negative_df = negative_df[negative_df.negative_news < 0]
negative_df = negative_df.groupby(['subreddit']).count().reset_index()
negative_df.head()
stats = pd.merge(positive_df,negative_df, how='inner',on=['subreddit','subreddit'])

In [5]:
import hvplot.pandas
import matplotlib.pyplot as plt

%matplotlib inline
stats.hvplot.bar(x="subreddit", y=["positive_news", "negative_news"],
                 stacked=True,rot=90,height=400, width=800,
                xlabel='CryptoCurrencies',ylabel='news count', legend='top',
                title='Reddit News Distribution for CryptoCurrencies')

:Bars   [subreddit,Variable]   (value)